## Test orc file

In [60]:
import pandas as pd
txt_path = '/Users/ymdt/src/dreem/test/test_files/bitvector/mysample/temp/AGGATTAG.txt'
output_path = '/Users/ymdt/src/dreem/test/test_files/bitvector/mysample/AGGATTAG.orc'
def make_bv_orc_from_txt(txt_path, output_path):
    with open(txt_path, 'r') as f:
        line = f.readline().split('\t')
        ref_seq = line[2]
    df = pd.read_csv(txt_path, sep='\t', skiprows=2).set_index('Query_name')
    df.drop('N_Mutations', axis=1, inplace=True)
    df = df.join(df['Bit_vector'].str.split('', expand=True)).drop(['Bit_vector',0,201], axis=1)
    df.columns = [str(c)+str(i) for i,c in enumerate(ref_seq)]
    df.to_orc(output_path, index=False)     

df = pd.read_orc(output_path)
df

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ymdt/src/dreem/test/test_files/bitvector/mysample/AGGATTAG.orc'

## Test csv file

In [ ]:
import sys, os
sys.path.append('/Users/ymdt/src/dreem/test')
from config import *
from generate_test_files import *
import pandas as pd
import numpy as np

cols_types = {
    'sample':str,
    'construct': str, 
    'cluster': int,
    'sequence':str,
    'num_reads': int,
    'num_aligned': int,
    'num_of_mutations': object,
    'mut_bases': object,
    'info_bases': object,
    'cov_bases': object,
    'del_bases': object,
    'ins_bases': object,
    'mod_bases_A': object,
    'mod_bases_C': object,
    'mod_bases_G': object,
    'mod_bases_T': object,
    'mut_rates': object,
    'worst_cov_bases': int,
    'skip_short_reads': int,
    'skip_too_many_muts': int,
    'skip_low_mapq': int
    }

df = pd.DataFrame({k:pd.Series([], dtype=v) for k,v in cols_types.items()}, index=np.arange(10))

df['sample'] = SAMPLE_NAME
df['construct'] = CONSTRUCTS
df['sequence'] = [build_sequence_from_barcode(barcode, SEQUENCE, BARCODE_START, BARCODE_END) for barcode in BARCODES]
df['worst_cov_bases'] = NUM_READS
df['skip_short_reads'] = 0
df['skip_too_many_muts'] = 0
df['skip_low_mapq'] = 0

lib = pd.read_csv('/Users/ymdt/src/dreem/test/test_files/library.csv', index_col=0).reset_index().rename(columns={'section_name':'section'})
df = df.merge(lib, on='construct', how='left')
samples = pd.read_csv('/Users/ymdt/src/dreem/test/test_files/samples.csv', index_col=0).reset_index()
df = df.merge(samples, on='sample', how='left')

reads_array = np.zeros((NUM_READS*2,LENGTH_SEQUENCE))
for read in [0,1]:
    for r in range(NUM_READS):
        reads_array[r+read*NUM_READS] = add_mutation(r, np.zeros(LENGTH_SEQUENCE, dtype=int))
print(reads_array.sum(axis=1))

num_reads = {
    'pop_avg': NUM_READS,
    '1': NUM_READS//2,
    '2': NUM_READS//2 - NUM_READS//10,
    '3': NUM_READS//10,
}
mutation_points = MUTATION_THRESHOLD.values()

num_of_mutations = {
    'pop_avg':np.zeros(len(SEQUENCE), dtype=int),
    '1': np.zeros(len(SEQUENCE), dtype=int),
    '2': np.zeros(len(SEQUENCE), dtype=int),
    '3':np.zeros(len(SEQUENCE), dtype=int),
}
for k, v in num_of_mutations.items():
    num_of_mutations[k][1] = num_reads[k]

mut_bases = {
    'pop_avg':np.zeros(len(SEQUENCE), dtype=int),
    '1': np.zeros(len(SEQUENCE), dtype=int),
    '2': np.zeros(len(SEQUENCE), dtype=int),
    '3':np.zeros(len(SEQUENCE), dtype=int),
}


df_out = pd.DataFrame()
for cluster in ['pop_avg','1','2','3']:
    df_temp  = df.copy()
    df_temp['cluster'] = cluster
    df_temp['mut_bases'] = df_temp['mut_bases'].astype(object)
    df_temp['num_reads'], df_temp['num_aligned'] = num_reads[cluster], num_reads[cluster]
    df_temp['num_of_mutations'] = [num_of_mutations[cluster]]*len(df_temp)
    df_out = pd.concat([df_out, df_temp])
df = df_out.reset_index(drop=True)
df.to_csv('/Users/ymdt/src/dreem/test/test_files/'+SAMPLE_NAME+'.csv', index=False)

df



KeyError: 0

### test demultiplexing module

In [5]:
import sys, os
import pandas as pd
import numpy as np

path = '/Users/ymdt/src/dreem/test/test_files/input/demultiplexing/test_set_1/'

def hamming_distance(s1, s2):
    assert len(s1) == len(s2)
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

def reverse_complement(seq):
    return seq[::-1].translate(str.maketrans('ATCG', 'TAGC'))

def barcode_in_read(barcode, read, max_hamming_distance=1):
    if len(read) < len(barcode):
        return False
    for i in range(len(read)-len(barcode)+1):
        if hamming_distance(barcode, read[i:i+len(barcode)]) <= max_hamming_distance:
            return True
    return False

def __demultiplex(f1, f2, library, output_folder, temp_folder):
    """Demultiplex a pair of FASTQ files."""
    
    library = pd.read_csv(library)
    f1 = pd.DataFrame(np.loadtxt(f1, dtype=str).reshape(-1, 4), columns=['rname','rseq','+','qual'])
    f2 = pd.DataFrame(np.loadtxt(f2, dtype=str).reshape(-1, 4), columns=['rname','rseq','+','qual'])
    
    print(f1.shape, f2.shape)
    print(output_folder)
    
    for df in [f1,f2]:
        df['construct'] = df['rname'].str.split(':').str[0].str[1:]
        
    constructs = library['construct'].unique()
    barcodes = library['barcode'].unique()

    assert len(constructs) == len(barcodes)
    for construct, barcode in zip(constructs, barcodes):
        # chec if the barcode andthe construct are on the same row
        assert library.loc[library['construct']==construct, 'barcode'].values[0] == barcode

    output_folder = os.path.join(output_folder,'output','demultiplexing','test_set_1')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # copy the reads from the fastq files that contain the barcode in a fastq file named after the construct in the output folder
    for primer in [1,2]:
        for construct, barcode in zip(constructs, barcodes):
            if primer == 2:
                barcode = reverse_complement(barcode)
            
            df = f1 if primer == 1 else f2
            
            with open(output_folder+'/'+construct+'_R'+str(primer)+'.fastq', 'w') as f:
                for _, r in df.iterrows():
                    if barcode_in_read(barcode, r['rseq']):
                        f.write(r['rname']+'\n'+r['rseq']+'\n+\n'+r['qual']+'\n')
    return 1

__demultiplex('/Users/ymdt/src/dreem/test/test_files/input/demultiplexing/test_set_1/test_set_1_R1.fastq', \
                '/Users/ymdt/src/dreem/test/test_files/input/demultiplexing/test_set_1/test_set_1_R2.fastq',\
                   '/Users/ymdt/src/dreem/test/test_files/input/demultiplexing/test_set_1/library.csv',\
                   '/Users/ymdt/src/dreem/test/test_files',
                   ''\
)
                   

(20, 4) (20, 4)
/Users/ymdt/src/dreem/test/test_files


1

In [6]:
def my_function(school, standard, city, name):
    schoolName  = school
    cityName = city
    standardName = standard
    studentName = name
    
data = {'school':'DAV', 'standard': '7', 'name': 'abc', 'city': 'delhi'}

my_function(**data)

In [9]:
import pandas as pd
import dreem.util as util

#from click_option_group import optgroup, RequiredMutuallyExclusiveOptionGroup
import click
import os
from dreem.demultiplexing.demultiplexing import demultiplex


def run(**args):
    """Run the demultiplexing pipeline.

    Demultiplexes the reads and outputs one fastq file per construct in the directory `output_path`, using `temp_path` as a temp directory.

    Parameters from args:
    -----------------------
    library: str
        Path to the library file. Columns are (non-excusively): ['construct', 'barcode_start', 'barcode']
    fastq1: str
        Path to the FASTQ file or list of paths to the FASTQ files, forward primer.
    fastq2: str
        Path to the FASTQ file or list of paths to the FASTQ files, reverse primer.
    out_dir: str
        Name of the output directory.

    Returns
    -------
    1 if successful, 0 otherwise.

    """
    # Get the paths
    root = args['out_dir']
    temp_folder = os.path.join(root,'temp','demultiplexing')
    output_folder = os.path.join(root,'output','demultiplexing')
    fastq1 = args['fastq1'] if type(args['fastq1']) == list else [args['fastq1']]
    fastq2 = args['fastq2'] if type(args['fastq2']) == list else [args['fastq2']]

    # Make the folders
    util.make_folder(output_folder)
    util.make_folder(temp_folder)

    # Demultiplex
    for f1 in fastq1:
        for f2 in fastq2:
            if f1[:-len('_R1.fastq')] == f2[:-len('_R2.fastq')]:
                sample = f1.split('/')[-1][:-len('_R1.fastq')]
                util.make_folder(os.path.join(output_folder, sample))
                assert demultiplex(f1, f2, args['library'], os.path.join(output_folder, sample), os.path.join(temp_folder, sample)), "Demultiplexing failed"
    return 1


if __name__ == '__main__':
    path_input = os.path.join(os.getcwd(),'test','test_files','input','demultiplexing')
    path_predicted = os.path.join(os.getcwd(),'test','test_files','predicted_output','demultiplexing')
    path_output = os.path.join(os.getcwd(),'test','test_files')
    sample = 'test_set_1'
    args = {
        'fastq1': '{}/{}_R1.fastq'.format(os.path.join(path_input,sample),sample),
        'fastq2': '{}/{}_R2.fastq'.format(os.path.join(path_input,sample),sample),
        'library': '{}/library.csv'.format(os.path.join(path_input,sample)),
        'out_dir': path_output
    }
    print(args)
    run(**args)

{'fastq1': '/Users/ymdt/src/dreem/test/test_files/input/demultiplexing/test_set_1/test_set_1_R1.fastq', 'fastq2': '/Users/ymdt/src/dreem/test/test_files/input/demultiplexing/test_set_1/test_set_1_R2.fastq', 'library': '/Users/ymdt/src/dreem/test/test_files/input/demultiplexing/test_set_1/library.csv', 'out_dir': '/Users/ymdt/src/dreem/test/test_files'}
